# Mini Git - Branching and Merging Demo

This notebook demonstrates branching and merging operations.

## Data Structures Used
- **Directed Acyclic Graph (DAG)**: Branch relationships and commit graph
- **Hash Table**: Branch name to commit mapping
- **Graph Traversal**: Finding common ancestors

In [1]:
# Clean start
import shutil
import os

test_dir = './test_branching'
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
print('Ready for fresh demo')

Ready for fresh demo


In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from src.repository import Repository
import shutil

## Setup Test Repository

In [3]:
# Clean start
test_dir = './test_branching'
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
os.makedirs(test_dir)

# Initialize
repo = Repository(test_dir)
repo.init(author='Branching Demo')
print("✓ Initialized repository")

# Create initial commit
file1 = os.path.join(test_dir, 'main.txt')
with open(file1, 'w') as f:
    f.write('Main branch content\n')

repo.add(['main.txt'])
commit1 = repo.commit("Initial commit on main")
print(f"✓ Created initial commit: {commit1[:8]}")

✓ Initialized repository
✓ Created initial commit: ddc66e5f


## 1. Create Branch

**DAG Structure**: Each branch is a node pointing to a commit.

In [4]:
# Create feature branch
if repo.branch('feature'):
    print("✓ Created branch: feature")
    
    # Show branch structure
    branches = repo.branch()
    current = repo.branch_manager.get_current_branch()
    
    print("\nBranch Graph:")
    for branch in branches:
        branch_obj = repo.branch_manager.get_branch(branch)
        commit_hash = branch_obj.commit_hash
        marker = '*' if branch == current else ' '
        print(f"{marker} {branch} -> {commit_hash[:8]}")
else:
    print("Failed to create branch")

✓ Created branch: feature

Branch Graph:
  feature -> ddc66e5f
* main -> ddc66e5f


## 2. Switch Branch (Checkout)

**O(1) Operation**: Hash table lookup to switch branches.

In [5]:
# Switch to feature branch
if repo.checkout('feature'):
    print("✓ Switched to branch: feature")
    print(f"  Current branch: {repo.branch_manager.get_current_branch()}")
    print(f"  Commit: {repo.branch_manager.get_current_commit()[:8]}")

✓ Switched to branch: feature
  Current branch: feature
  Commit: ddc66e5f


## 3. Make Changes on Feature Branch

In [6]:
# Add feature file
feature_file = os.path.join(test_dir, 'feature.txt')
with open(feature_file, 'w') as f:
    f.write('New feature implementation\n')

repo.add(['feature.txt'])
commit2 = repo.commit("Add feature")
print(f"✓ Created commit on feature: {commit2[:8]}")

# Show branch divergence
print("\nBranch Graph (after feature commit):")
for branch in repo.branch():
    branch_obj = repo.branch_manager.get_branch(branch)
    commit_hash = branch_obj.commit_hash
    current = repo.branch_manager.get_current_branch()
    marker = '*' if branch == current else ' '
    print(f"{marker} {branch} -> {commit_hash[:8]}")

print("\nNote: feature and main now point to different commits!")

✓ Created commit on feature: cbe8bff8

Branch Graph (after feature commit):
* feature -> cbe8bff8
  main -> ddc66e5f

Note: feature and main now point to different commits!


## 4. Make Changes on Main Branch

In [7]:
# Switch back to main
repo.checkout('main')
print(f"✓ Switched to: {repo.branch_manager.get_current_branch()}")

# Modify main
with open(file1, 'a') as f:
    f.write('Additional main branch content\n')

repo.add(['main.txt'])
commit3 = repo.commit("Update main branch")
print(f"✓ Created commit on main: {commit3[:8]}")

✓ Switched to: main
✓ Created commit on main: 4e305997


## 5. Visualize Commit Graph (DAG)

The commit history forms a Directed Acyclic Graph.

In [8]:
print("Commit DAG:")
print("="*60)

# Show main branch history
print("\nmain branch:")
main_commits = repo.log()
for i, commit in enumerate(main_commits):
    indent = "  " * i
    print(f"{indent}{commit.hash[:8]} - {commit.message}")
    if commit.parent_hash:
        print(f"{indent}  ↓")

# Show feature branch history
repo.checkout('feature')
print("\nfeature branch:")
feature_commits = repo.log()
for i, commit in enumerate(feature_commits):
    indent = "  " * i
    print(f"{indent}{commit.hash[:8]} - {commit.message}")
    if commit.parent_hash:
        print(f"{indent}  ↓")

repo.checkout('main')

Commit DAG:

main branch:
4e305997 - Update main branch
  ↓
  ddc66e5f - Initial commit on main

feature branch:
cbe8bff8 - Add feature
  ↓
  ddc66e5f - Initial commit on main


True

## 6. Find Common Ancestor

**Graph Traversal Algorithm**: Used for merge operations.

In [9]:
# Get commit hashes
main_commit = repo.branch_manager.get_branch('main').commit_hash
feature_commit = repo.branch_manager.get_branch('feature').commit_hash

print(f"Main commit:    {main_commit[:8]}")
print(f"Feature commit: {feature_commit[:8]}")

# Find common ancestor using graph traversal
common_ancestor = repo.commit_history.find_common_ancestor(main_commit, feature_commit)

if common_ancestor:
    print(f"\n✓ Common ancestor: {common_ancestor[:8]}")
    ancestor_commit = repo.commit_history.get_commit(common_ancestor)
    print(f"  Message: {ancestor_commit.message}")
else:
    print("No common ancestor found")

Main commit:    4e305997
Feature commit: cbe8bff8

✓ Common ancestor: ddc66e5f
  Message: Initial commit on main


## 7. Merge Branches

**Three-Way Merge Algorithm**: Uses common ancestor as base.

In [10]:
# Ensure we're on main
repo.checkout('main')

# Merge feature into main
print("Merging feature into main...\n")
result = repo.merge('feature')

if result['success']:
    print("✓ Merge successful!")
    print(f"  {result.get('message', '')}")
    if 'files' in result:
        print(f"  Merged files: {', '.join(result['files'])}")
else:
    print("✗ Merge failed!")
    print(f"  Error: {result.get('error', 'Unknown')}")
    if 'conflicts' in result:
        print("  Conflicts in:")
        for path in result['conflicts']:
            print(f"    - {path}")

Merging feature into main...

✗ Merge failed!
  Error: Unknown
  Conflicts in:
    - main.txt


## 8. Branch Performance Analysis

Demonstrate hash table efficiency for branch operations.

In [11]:
import time

# Create multiple branches
num_branches = 100
current_commit = repo.branch_manager.get_current_commit()

start = time.time()
for i in range(num_branches):
    repo.branch_manager.create_branch(f'test_branch_{i}', current_commit)
end = time.time()

print(f"Created {num_branches} branches in {(end-start)*1000:.2f}ms")
print(f"Average per branch: {((end-start)/num_branches)*1000:.4f}ms")

# Test lookup speed
start = time.time()
for i in range(1000):
    branch = repo.branch_manager.get_branch('test_branch_50')
end = time.time()

print(f"\n1000 branch lookups in {(end-start)*1000:.2f}ms")
print("✓ Demonstrates O(1) hash table lookup")

# Total branches
all_branches = repo.branch_manager.list_branches()
print(f"\nTotal branches: {len(all_branches)}")

Created 100 branches in 0.09ms
Average per branch: 0.0009ms

1000 branch lookups in 0.15ms
✓ Demonstrates O(1) hash table lookup

Total branches: 102


## Summary

This demo showed:
1. **DAG Structure**: Branches as nodes in a commit graph
2. **Hash Table**: O(1) branch operations
3. **Graph Traversal**: Finding common ancestors for merges
4. **Three-Way Merge**: Using base, ours, and theirs

The branch and merge operations are efficient and scale well!

In [12]:
# Cleanup
# shutil.rmtree(test_dir)  # Uncomment to clean up
print("Demo complete!")

Demo complete!
